In [1]:
import sys
sys.path.insert(0, './../../../Models')
from sphere_points import generate_points

import numpy as np
np.random.seed(0)
import matplotlib.pyplot as plt

import torch
torch.manual_seed(0)
import torch.nn as nn
from tqdm import tqdm
from torch.optim import SGD
from torch.nn.functional import normalize, one_hot
# import torch.nn.functional as F
import torchvision
from torchvision import transforms
# from torch.optim.lr_scheduler import StepLR
from torch.utils.data import DataLoader

In [2]:
# device = torch.device("cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu")
# device = torch.device("cpu")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [3]:
def plot_losses(losses):
    losses = np.array(losses)
    n_dims, epochs = losses.shape
    plt.figure(figsize = (12, 5))
    for l in range(n_dims):
        plt.subplot(1, n_dims, l + 1)
        plt.plot(1 + np.arange(epochs), losses[l])
        plt.title(f"Layer {l + 1} Loss")
        plt.xlabel("Epochs")
        plt.ylabel("Loss")
    plt.tight_layout()
    plt.show()

In [4]:
# model_loss = nn.CrossEntropyLoss()
num_classes = 10

In [5]:
def classifier_head_train(inp_embedding, classifier_weights, labels):
    # model_loss = nn.CrossEntropyLoss()
    inp_embedding = normalize(inp_embedding, p=2, dim=-1)
    classifier_output = torch.mm(inp_embedding, classifier_weights)
    # classifier_output = 1 - (torch.acos(classifier_output)/np.pi)
    classifier_output = classifier_output * one_hot(labels, num_classes = num_classes).type(torch.float32)
    # classifier_output = torch.softmax(classifier_output, dim=-1)
    # loss = model_loss(classifier_output, one_hot(labels, num_classes = num_classes).type(torch.float32))
    theta = 1
    loss = torch.mean(torch.log(2 - (theta * torch.sum(classifier_output,1))))
    return loss

In [6]:
def classifier_head(inp_embedding, classifier_weights, labels):
    inp_embedding = normalize(inp_embedding, p=2, dim=-1)
    classifier_output = torch.mm(inp_embedding, classifier_weights)
    # classifier_output = 1 - (torch.acos(classifier_output)/np.pi)
    # classifier_output = torch.softmax(classifier_output, dim=-1)
    # loss = model_loss(classifier_output, one_hot(labels, num_classes = num_classes).type(torch.float32))
    return torch.argmax(classifier_output, dim=1).tolist()

In [7]:
initial = None
# num_classes = 10

# Data dimension
# (num_data, num_features) => no dimension for batch size please
class Layer(nn.Linear):
    def __init__(self, in_features, out_features, bias, device, lr, apply_dropout=False):
        super().__init__(in_features, out_features, bias, device)
        self.out_features = out_features
        self.bias_flag = bias
        self.lr = lr
        self.num_classes = num_classes
        self.dimension = out_features
        # self.activation = DyT(1)
        # self.activation = nn.LeakyReLU(negative_slope=0.001)
        # self.leaky_relu = nn.LeakyReLU(negative_slope=0.001)
        # self.activation = nn.GELU()
        # self.opt = Adam(self.parameters(), lr = self.lr)
        # self.opt = SGD(self.parameters(), lr=self.lr, momentum=0.9)
        # self.scheduler = torch.optim.lr_scheduler.MultiStepLR(self.opt, 
        #                                                       milestones=[60], gamma=0.1)
        # nn.init.kaiming_normal_(self.weight, mode='fan_in')
        
        fc1_limit = np.sqrt(6.0 / in_features)
        torch.nn.init.uniform_(self.weight, a=-fc1_limit, b=fc1_limit)
        
        self.dropout = nn.Dropout(0.1)
        self.apply_dropout = apply_dropout
        global initial
        self.directions = generate_points(self.num_classes, self.dimension, steps = 10000)
        initial = np.array(self.directions)
        self.directions = [torch.tensor(t, dtype = torch.float32).to(device) for t in self.directions]
        self.direction_weights = torch.zeros((len(self.directions[0]), len(self.directions)), device=device, 
                                             requires_grad=False)
        for i in range(len(self.directions)):
            self.direction_weights[:, i] = normalize(self.directions[i], p = 2, dim=-1)

    def train(self, x, labels):
        if self.apply_dropout:
            x = self.dropout(x)
        # activation = nn.ELU()
        activation = nn.LeakyReLU(negative_slope=0.001)
        # activation = nn.Tanh
        # activation = self.activation
        opt = SGD(self.parameters(), lr=self.lr)
        # activation = nn.ReLU()
        # opt = Adam(self.parameters(), lr=self.lr)
        # print(f"gpu used {torch.cuda.max_memory_allocated(device=None)} memory")
        if self.bias_flag:
            y = activation(torch.mm(x, self.weight.T) + self.bias.unsqueeze(0))
        else:
            y = activation(torch.mm(x, self.weight.T))
        # if self.apply_dropout:
        #     x = self.dropout(x)
        # y = self.forward(x) # shape: (num_data, out_features)
        '''
        y = normalize(y, p = 2, dim = 1)
        '''
        # import pdb;pdb.set_trace()
        loss = classifier_head_train(y, self.direction_weights, labels)
        
        '''
        directions = torch.zeros_like(y)
        for i in range(y.shape[0]):
            directions[i, :] = self.directions[label[i]]
        
        loss = loss_layer(y, directions)
        '''
        opt.zero_grad(set_to_none=True)
        loss.backward(retain_graph = False)
        opt.step()
        # self.scheduler.step()
        
        # normalize the directions
        # self.directions.data = normalize(self.directions.data, p = 2, dim = 1)

        return loss.detach().item(), y
    
    def test(self, x, labels):
        with torch.no_grad():
            # activation = nn.ELU()
            # y = self.forward(x)
            activation = nn.LeakyReLU(negative_slope=0.001)
            # activation = nn.ReLU()
            # activation = nn.Tanh()
            # activation = self.activation
            if self.bias_flag:
                y = activation(torch.mm(x, self.weight.T) + self.bias.unsqueeze(0))
            else:
                y = activation(torch.mm(x, self.weight.T))
        max_idx_list =[]
        max_idx_list = classifier_head(y, self.direction_weights, labels)
        '''
        for dat in range(y.shape[0]):
            max = -np.inf
            max_idx = 0
            for i in range(self.num_classes):
                cos_sim = cos_similarity(y[dat, :].unsqueeze(0), self.directions[i].reshape(1, -1))
                if cos_sim > max:
                    max = cos_sim
                    max_idx = i
            max_idx_list.append(max_idx)
        '''
        return torch.tensor(max_idx_list, device=device), y

In [8]:
class Net(nn.Module):
    def __init__(self, dims_list, bias, epochs, lr, device):
        super(Net, self).__init__()
        self.dims_list = dims_list
        self.bias = bias
        self.epochs = epochs
        self.lr = lr
        self.device = device
        self.layers = []
        # self.sigmoid = nn.Sigmoid()
        global initial
        for d in range(len(self.dims_list) - 1):
            print(f"Initialization {d + 1} / {len(self.dims_list) - 1}")
            self.layers += [Layer(self.dims_list[d], self.dims_list[d + 1], self.bias, self.device, self.lr)]
            print("Complete\n")
        
    def train(self, train_loader, test_loader):
        layer_loss_list = []
        acc_train = []
        acc_test = []
        layer_w = [[] for _ in range(len(self.dims_list) - 1)]
        
        for i in range(len(self.layers)):
            layer_loss_list.append([])
        pbar = tqdm(total = self.epochs * len(train_loader) * len(self.layers), 
                    desc = f"Training", position = 0, leave = True)
        
        # Test the network
        with torch.no_grad():
           
            for i in range(len(self.layers)):
                layer_w[i].append(torch.norm(self.layers[i].weight, p=2).item())
            acc_train.append(net.test(trainloader))
            acc_test.append(net.test(testloader))
        
        for epoch in range(self.epochs):

            if epoch and not (epoch % 10): 
                # learning rate decay
                for i in range(len(self.layers)):
                    self.layers[i].lr = self.layers[i].lr - 0.1
                    print('lr decreased to ', self.layers[i].lr)
            
            loss_agg = [0] * len(self.layers)
            for dat in train_loader:
                x, label = dat
                x = x.to(device)
                label = label.to(device)
                for i in range(len(self.layers)):
                    
                    loss, y = self.layers[i].train(x, label)
                    self.layers[i].zero_grad(set_to_none=True)
                    x = y.detach()
                    loss_agg[i] += loss / len(train_loader)
                    del y
                    pbar.update(1)
            pbar.set_postfix(epoch = epoch + 1, loss = loss_agg)
            for i in range(len(self.layers)):
                layer_loss_list[i].append(loss_agg[i])
                
            # Test the network
            with torch.no_grad():
                for i in range(len(self.layers)):
                    layer_w[i].append(torch.norm(self.layers[i].weight, p=2).item())
                acc_train.append(net.test(train_loader))
                acc_test.append(net.test(test_loader))  
                
        pbar.close()
        return [layer_loss_list, acc_train, acc_test, layer_w]
    
    def test(self, data_loader):
        all_accuracy = []
        # correct = [0 for _ in range(len(self.layers) + 1)]
        # total = [0 for _ in range(len(self.layers) + 1)]
        correct = [0 for _ in range(len(self.layers))]
        total = [0 for _ in range(len(self.layers))]
        for dat in data_loader:
            x = dat[0]
            label = dat[1]
            x = x.to(device)
            label = label.to(device)
            num = label
            preds = []
            
            for i in range(len(self.layers)):
                pred, x = self.layers[i].test(x, label)
                preds.append(pred)
            
            for i in range(len(preds)):
                correct[i] += (preds[i] == num).sum().item()
                total[i] += num.shape[0]

            # m = np.array([p.cpu().numpy() for p in preds])
            # # from scipy import stats
            # # mode_pred = stats.mode(m)
            # # correct[-1] += (mode_pred[0].flatten() == num.cpu()).sum().item()
            # correct[-1] += (np.mean(m, axis = 0).astype(int).flatten() == num.cpu()).sum().item()
            # total[-1] += num.shape[0]
                
        all_accuracy.append(np.array(correct) / total[-1])
        return all_accuracy

In [9]:
flatten_transform = transforms.Lambda(lambda x: x.view(x.size(0), -1).squeeze())
'''
# Define data transformations
transform = transforms.Compose([
    transforms.ToTensor(),
    flatten_transform
])
'''

# # Define data transformations
# transform = transforms.Compose([
#     transforms.ToTensor(),
#     transforms.Normalize((0.,), (0.5,)),
#     flatten_transform
# ])

transform = transforms.Compose([transforms.ToTensor(), flatten_transform]) # this normalizes to [0,1]

trainset = torchvision.datasets.MNIST(root='./../../../Data', train=True, download=True, transform=transform)
testset = torchvision.datasets.MNIST(root='./../../../Data', train=False, download=True, transform=transform)
batch_size = 50
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=False)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle = False)


# full_data_set = torch.utils.data.ConcatDataset([trainset, testset])

In [10]:
# ratio = [0.01, 0.05, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
ratio = [0.2]
test_acc = []
train_acc = []
w_layers = []
loss = []

output_layer_size = [1024]

num_runs = 5

# for r in ratio:
    # print(f"train ratio: {r}")
'''    
test_size = int(r * len(full_data_set))
train_size = len(full_data_set) - test_size
train_dataset, test_dataset = torch.utils.data.random_split(full_data_set, [train_size, test_size])

# Create DataLoader for training set
trainloader = DataLoader(train_dataset, batch_size = 64, shuffle=True)

# Create DataLoader for test set
testloader = DataLoader(test_dataset, batch_size = 64, shuffle=False)
'''
for l2_size in output_layer_size:
    for _ in range(num_runs):
        
        # dims_list = [784, 1024, 924, 824, 724, 624, 524, 324, 224, l2_size]
        # dims_list = [784, 1000, 34]
        dims_list = [784, 1024, 1024, 1024, 1024, 1024, 1024, 1024, 1024, l2_size]
        bias = True
        epochs = 200
        lr = 2.5
        num_classes = 10
        net = Net(dims_list, bias, epochs, lr, device)
        
        # Train the network
        layer_loss_list = net.train(trainloader, testloader)
        
        # plot_losses(layer_loss_list[0])
        
        '''
        # Test the network
        acc_train = net.test(trainloader)
        print(f"Train accuracy: {acc_train * 100:.2f}%")
        
        acc_test = net.test(testloader)
        print(f"Test accuracy: {acc_test * 100:.2f}%")
        '''
        loss.append(layer_loss_list[0])
        train_acc.append(layer_loss_list[1])
        test_acc.append(layer_loss_list[2])
        w_layers.append(layer_loss_list[3])

Initialization 1 / 9
Complete

Initialization 2 / 9
Complete

Initialization 3 / 9
Complete

Initialization 4 / 9
Complete

Initialization 5 / 9
Complete

Initialization 6 / 9
Complete

Initialization 7 / 9
Complete

Initialization 8 / 9
Complete

Initialization 9 / 9
Complete



Training:   5%| | 108229/2160000 [03:58<14:39:29, 38.88it/s, epoch=10, loss=[0.3582771361867589, 0.2981972372035182, 0.2952516698588928, 0.28905420338114113, 0.2914081706106662, 0.2857880684981746, 0.2912

lr decreased to  2.4
lr decreased to  2.4
lr decreased to  2.4
lr decreased to  2.4
lr decreased to  2.4
lr decreased to  2.4
lr decreased to  2.4
lr decreased to  2.4
lr decreased to  2.4


Training:  10%| | 216127/2160000 [07:54<16:34:43, 32.57it/s, epoch=20, loss=[0.3526476955413817, 0.29080683658520345, 0.28452212286492207, 0.27954770406087237, 0.27809948215881974, 0.275966898947954, 0.28

lr decreased to  2.3
lr decreased to  2.3
lr decreased to  2.3
lr decreased to  2.3
lr decreased to  2.3
lr decreased to  2.3
lr decreased to  2.3
lr decreased to  2.3
lr decreased to  2.3


...

Training:  90%|▉| 1944208/2160000 [1:15:04<1:28:58, 40.42it/s, epoch=180, loss=[0.34399080408116195, 0.28152878810961995, 0.27152891064683626, 0.27592521178225676, 0.2909200218568246, 0.2739134452988707, 

lr decreased to  0.6999999999999988
lr decreased to  0.6999999999999988
lr decreased to  0.6999999999999988
lr decreased to  0.6999999999999988
lr decreased to  0.6999999999999988
lr decreased to  0.6999999999999988
lr decreased to  0.6999999999999988
lr decreased to  0.6999999999999988
lr decreased to  0.6999999999999988


Training:  95%|▉| 2052106/2160000 [1:19:17<52:56, 33.97it/s, epoch=190, loss=[0.34389630009730643, 0.2813922677685818, 0.27138084525863365, 0.27523945840696507, 0.2883856549610701, 0.2727192341784632, 0.2

lr decreased to  0.5999999999999989
lr decreased to  0.5999999999999989
lr decreased to  0.5999999999999989
lr decreased to  0.5999999999999989
lr decreased to  0.5999999999999989
lr decreased to  0.5999999999999989
lr decreased to  0.5999999999999989
lr decreased to  0.5999999999999989
lr decreased to  0.5999999999999989


Training: 100%|█| 2160000/2160000 [1:23:25<00:00, 431.53it/s, epoch=200, loss=[0.3438074507812659, 0.28127340219914887, 0.27125316590070736, 0.27351496992011864, 0.2860166090975205, 0.27202082378168907, 0


In [11]:
print(test_acc)
print(np.max(test_acc))

[[[array([0.0769, 0.0512, 0.1242, 0.0869, 0.0895, 0.0983, 0.089 , 0.1198,
       0.1524])], [array([0.8889, 0.9054, 0.9089, 0.9111, 0.911 , 0.9112, 0.9114, 0.9112,
       0.911 ])], [array([0.9   , 0.9163, 0.9186, 0.9193, 0.92  , 0.9204, 0.9204, 0.9203,
       0.9203])], [array([0.9044, 0.9223, 0.9246, 0.9242, 0.9259, 0.9264, 0.9266, 0.9267,
       0.927 ])], [array([0.907 , 0.9258, 0.9283, 0.9291, 0.9297, 0.9301, 0.9305, 0.9308,
       0.9308])], [array([0.9081, 0.9287, 0.9311, 0.9323, 0.9327, 0.9326, 0.9328, 0.9331,
       0.933 ])], [array([0.909 , 0.9305, 0.9332, 0.9339, 0.9349, 0.9349, 0.935 , 0.9349,
       0.9351])], [array([0.9096, 0.9323, 0.935 , 0.9367, 0.937 , 0.9371, 0.9367, 0.937 ,
       0.9365])], [array([0.9096, 0.9335, 0.9372, 0.9384, 0.9381, 0.9382, 0.9387, 0.9389,
       0.939 ])], [array([0.9102, 0.935 , 0.9383, 0.9393, 0.939 , 0.9391, 0.9397, 0.9397,
       0.9398])], [array([0.9107, 0.9366, 0.9396, 0.9399, 0.9396, 0.9404, 0.9408, 0.9411,
       0.9407])], [array([

In [12]:
print(train_acc)
print(np.max(train_acc))

[[[array([0.07681667, 0.05031667, 0.12      , 0.08858333, 0.10001667,
       0.10185   , 0.08891667, 0.12171667, 0.14408333])], [array([0.88228333, 0.8996    , 0.90306667, 0.9034    , 0.90338333,
       0.904     , 0.90436667, 0.90438333, 0.90423333])], [array([0.89298333, 0.91161667, 0.91438333, 0.91495   , 0.91513333,
       0.91531667, 0.91551667, 0.91528333, 0.91526667])], [array([0.89733333, 0.91783333, 0.92068333, 0.92111667, 0.92151667,
       0.92173333, 0.92185   , 0.92175   , 0.92185   ])], [array([0.90028333, 0.92241667, 0.92501667, 0.92558333, 0.9256    ,
       0.9259    , 0.92606667, 0.92621667, 0.92645   ])], [array([0.90245   , 0.92551667, 0.92826667, 0.92916667, 0.92941667,
       0.92965   , 0.92971667, 0.92978333, 0.9297    ])], [array([0.90395   , 0.92856667, 0.93103333, 0.93151667, 0.9319    ,
       0.93215   , 0.93213333, 0.93206667, 0.93223333])], [array([0.90466667, 0.93016667, 0.93298333, 0.93356667, 0.93388333,
       0.93411667, 0.93423333, 0.93423333, 0.934

In [13]:
np.save("./../new_data/mnist_train_acc_multilayer.npy", np.array(train_acc))
np.save("./../new_data/mnist_test_acc_multilayer.npy", np.array(test_acc))
np.save("./../new_data/w_mnist_bestconfig_multilayer.npy", np.array(w_layers))
np.save("./../new_data/loss_mnist_bestconfig_output_multilayer.npy", np.array(loss))